In [267]:
import numpy as np
import meshplot as mp
from skimage.measure import marching_cubes

In [268]:
from compas.geometry import Frame, Plane
from compas.geometry import Box, Sphere
from compas_vol.primitives import VolBox, VolSphere
from compas_vol.combinations import Union, SmoothUnion, Intersection
from compas_vol.modifications import Shell

In [286]:
#generate random points around a sphere
nbrPts = 10
sphereRad = 3.4
u = np.random.normal(size=nbrPts)
v = np.random.normal(size=nbrPts)
w = np.random.normal(size=nbrPts)
norm = (u*u + v*v + w*w)**(0.5)
pts = np.array([u/norm, v/norm, w/norm]).T * sphereRad

In [287]:
#create volumetric boxes and unify them
volBoxes = []
for [x,y,z] in pts:
    plane = Plane([x,y,z],[x,y,z])
    box = Box(Frame.from_plane(plane),1.2,1.2,3.5)
    vbox = VolBox(box, 0.1)
    volBoxes.append(vbox)
uvb = Union(volBoxes)

In [288]:
#combine core and branches
vs = VolSphere(Sphere([0,0,0], 2.0))
su = SmoothUnion(uvb, vs, 2.0)

In [289]:
#thicken shell and cull extremities
s = Shell(su, 0.2, 1.0)
node = Intersection(VolSphere(Sphere([0,0,0], 4.8)), s)

In [290]:
#workspace initialization
# lower and upper bounds
lbx, ubx = -5.0, 5.0
lby, uby = -5.0, 5.0
lbz, ubz = -5.0, 5.0
# resolution(s)
nx, ny, nz = 100, 100, 100
x, y, z = np.ogrid[lbx:ubx:nx*1j, lby:uby:ny*1j, lbz:ubz:nz*1j]
#voxel dimensions
gx = (ubx-lbx)/nx
gy = (uby-lby)/ny
gz = (ubz-lbz)/nz

In [291]:
dm = node.get_distance_numpy(x, y, z)

In [292]:
v, f, n, l = marching_cubes(dm, 0, spacing=(gx, gy, gz))
v += [lbx,lby,lbz]

In [293]:
p = mp.plot(v, f, c=np.array([0,0.57,0.82]), shading={"flat":False, "roughness":0.4, "metalness":0.01, "reflectivity":1.0})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1285529…